In [8]:
import os
import json
import pandas as pd
import traceback
import os

In [9]:
from langchain.chat_models import ChatOpenAI

In [10]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [11]:
KEY=os.getenv("OPENAI_API_KEY")

In [12]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.5) #from 0-2 you can mention values of temprature 

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain
import PyPDF2


In [14]:
#define the prompt template imput vairable and template 
TEMPLATE="""
Text:{text}
You are an expert mcq maker. Given the above text , it is your job to \
create a quiz of {number} multiple choice question for {subject} student in {tone} tone.
Make sure the question are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
## RESPONSE_JSON
{response_json}"""

## tone will decide the difficulty level


In [15]:
# Defining the output template
RESPONSE_JSON = {
    "1": {
        'mcq': "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        'mcq': "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        'mcq': "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "4": {
        'mcq': "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}


In [16]:
quize_generation_prompt = PromptTemplate(
    input_variable=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [17]:
quiz_chain=LLMChain(llm=llm,prompt=quize_generation_prompt,output_key="quiz",verbose=True)

e:\mcqgen\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
TEMPLATE2="""You are an expert english and writer. Given a Multiple choice Quiz for {subject} Student.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity if 
the quiz is not at per the congnitive and analytical abilities of the student,\
update the quiz question which needs to be changed and change the tone such that it perfectly fits the students ability
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:
"""

#template for the correct question pickup


In [19]:
quiz_evaluate_prompt=PromptTemplate(input_variable=['subject','quiz'],template=TEMPLATE)
#Creating a new template and chreating a chain for this chain

In [20]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluate_prompt,output_key='review',verbose=True)
#creating the review chain
#now connect both the chains- quiz chain and review chain

In [21]:
#using sequential chain to create the and combined chain
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','response_json'], 
                                          output_variables=['subject','quiz'],verbose=True)

In [22]:
#first we connected to tempalated using llmchains 
# second we are connecting two chains using sequential chains

In [23]:
file_path='E:\mcqgen\data.txt'

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\anura\AppData\Local\Temp\ipykernel_14240\2058488807.py:1: SyntaxWarning: invalid escape sequence '\m'
  file_path='E:\mcqgen\data.txt'


In [24]:
#created the data.txt file with mchine learning wiki data data
#reading the data using data loads
with open(file_path,'r') as file:
    TEXT = file.read()


In [25]:
file_path

'E:\\mcqgen\\data.txt'

In [26]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFrom a the

In [27]:
#serialize the python dictionary into a JSON-formating string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [28]:
NUMBER=3
SUBJECT="Machine Learning"
TONE='simple'

In [29]:

# Use the variables in your function call
with get_openai_callback() as cb:
    # How to setup token usages tack record we are using get openai callback
    response=generate_evaluate_chain(
        {
            'text': TEXT,
            'number': NUMBER,
            'subject': SUBJECT,
            'tone': TONE,
            'response_json': json.dumps(RESPONSE_JSON)
        }
    )

e:\mcqgen\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of

In [30]:
# to know total number of inputs and output tokens
print(f"Total Token:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Token:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}") #in dollars

Total Token:2753
Prompt Tokens:2298
Completion Token:455
Total Cost:0.004357


In [31]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nF

In [32]:
quiz=response.get("quiz")

In [33]:
quiz=json.loads(quiz)

In [34]:
#Create the databse format how to store the data
quiz_table_data=[]
for key,value in quiz.items():
    mcq = value["mcq"]
    options=" | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        
        ]
    )
correct = value["correct"]
quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [35]:
quiz_table_data

[{'MCQ': "What was the purpose of the experimental 'learning machine' Cybertron developed in the early 1960s?",
  'Choices': 'a: To analyze weather patterns | b: To recognize speech patterns | c: To play checkers | d: To study human cognitive processes',
  'Correct': 'b'}]

In [36]:
#converting into dataframe
quiz = pd.DataFrame(quiz_table_data)

In [37]:
quiz 

,MCQ,Choices,Correct
0,What was the purpose of the experimental 'lear...,a: To analyze weather patterns | b: To recogni...,b


In [38]:
quiz.to_csv("machinelearning,csv")